In [1]:
import pandas as pd
import  math                                                                             
from collections import defaultdict
from collections import Counter
import numpy as np
from tqdm import tqdm
from geopy.distance import geodesic as GD
import os
import random
import networkx as nx
import time
save_path = './Trajectory_Reconstruction/data_experiment'
E_data_name = 'yancheng'
data_path = '{}/{}/RN_model_data/roadnet'.format(save_path,E_data_name)
save_path_map = '{}/{}'.format(save_path,E_data_name)
save_path_rn = '{}/{}/RN_model_data'.format(save_path,E_data_name)

# 1 Obtain a graph whose edges are nodes

In [2]:
rn_dir =  '{}/{}osm1'.format(save_path_map,E_data_name)
g = nx.read_shp(rn_dir, simplify=True, strict=False)

/DATA2/lvxiaoling/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: read_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  


In [3]:
len(g.edges()) # yancheng

50909

In [4]:
for n, data in g.nodes(data=True):
    print(n,data)
    break

(120.5138333, 32.6580336) {'osmid': 679120638, 'y': 32.6580336, 'x': 120.5138333, 'ref': '88', 'highway': 'motorway_junction', 'street_cou': 3, 'ShpName': 'nodes'}


In [5]:
raw_uid = []
for u, v, data in g.edges(data=True):
    raw_uid.append(data['u'])
    raw_uid.append(data['v'])
a = Counter(raw_uid)
a = dict(sorted(a.items()))
raw2new_uid = {key:i for i,key in enumerate(a.keys())}

features = []
lat_lng = []

node2eid = {}
for edgeNum,(u, v, data) in enumerate(g.edges(data=True)):
    u_id = raw2new_uid[data['u']]
    v_id = raw2new_uid[data['v']]
    node2eid[(u_id,v_id)] = edgeNum+1
   
    data_lng_lat = data['Wkt'][11:][1:-1].split(',')
    single_lat_lng = []
    for k in data_lng_lat:
        k = k.strip()
        single_lat_lng.append((float(k.split(' ')[1]),float(k.split(' ')[0])))
    lat_lng.append(single_lat_lng)
    features.append(data['highway'])

In [6]:
fea = []
a = Counter(features)
a = dict(sorted(a.items()))
code_dict = {key:i for i,key in enumerate(a.keys())}
for value in features:
    fea.append([code_dict[value]])

In [7]:
def create_graph(nodes,  node_attrs={}, is_directed=True):
    g = nx.DiGraph()
    # add nodes
    for node in nodes:
        g.add_node(node, **node_attrs.get(node, node_attrs))
    return g
# example usage
nodes = [ i for i in range(len(g.edges()))]
node_attrs = {s_node:{'eid': i+1,'zone_area':((np.min([sv[1] for sv in lat_lng[i]]),np.min([ sv[0] for sv in lat_lng[i]]),np.max([ sv[1] for sv in lat_lng[i]]),np.max([ sv[0] for sv in lat_lng[i]]))),'coords':lat_lng[i],"features":fea[i]} for i,s_node in enumerate(nodes)}
edge_g = create_graph(nodes,node_attrs=node_attrs)

In [8]:
nx.write_gpickle(edge_g, '{}/{}_graph.gpickle'.format(save_path_map,E_data_name))
edge_g = nx.read_gpickle('{}/{}_graph.gpickle'.format(save_path_map,E_data_name))

In [9]:
len(edge_g.nodes())

50909

In [10]:
lat = [ single_value[0] for value in lat_lng for single_value in value]
lng = [ single_value[1] for value in lat_lng for single_value in value]
print('{}:zone_range = [{}, {}, {}, {}]'.format(E_data_name,min(lat),min(lng),max(lat),max(lng)))

yancheng:zone_range = [32.5928664, 119.455292, 34.4636842, 120.9120622]


# 2  get roadnet edgeOSM.txt and wayTypeOSM.txt

In [32]:
rn_dir =  '{}/{}osm1'.format(save_path_map,E_data_name)
g = nx.read_shp(rn_dir, simplify=True, strict=False)

/home/jnli/.conda/envs/RNTrajRec/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: read_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  


In [33]:
# 2.1  get edge osm

In [34]:
output_file = '{}/roadnet/edgeOSM.txt'.format(save_path_rn)

raw_uid = []
lat_lng = []
features = []
for u, v, data in g.edges(data=True):
    raw_uid.append(data['u'])
    raw_uid.append(data['v'])
a = Counter(raw_uid)
a = dict(sorted(a.items()))  # 按键排列
raw2new_uid = {key: i for i, key in enumerate(a.keys())}

with open(output_file, 'w') as file:
    for edgeNum, (u, v, data) in enumerate(g.edges(data=True)):
        u_id = raw2new_uid[data['u']]
        v_id = raw2new_uid[data['v']]

        data_lng_lat = data['Wkt'][11:][1:-1].split(',')
        num = len(data_lng_lat)
        single_lat_lng = []
        for k in data_lng_lat:
            k = k.strip()
            single_lat_lng.append(float(k.split(' ')[1]))
            single_lat_lng.append(float(k.split(' ')[0]))

        file.write(f"{edgeNum} {u_id} {v_id} {num} {' '.join(map(str, single_lat_lng))}\n")

        lat_lng.append(single_lat_lng)   
        features.append(data['highway'])

In [35]:
# 2.2  get way type 

In [36]:
features_code = []
a = Counter(features)
a = dict(sorted(a.items()))
code_dict = {key:i for i,key in enumerate(a.keys())}
features_code = [code_dict[value] for value in features]
len(Counter(features_code))

32

In [37]:
output_file = '{}/roadnet/wayTypeOSM.txt'.format(save_path_rn)
with open(output_file, 'w') as file:
    for index, value in enumerate(features):
        file.write(f"{index} {value} {features_code[index]}\n")

# 3 save the trajectory

## 3.1 process the raw data

In [42]:
df_traj = pd.read_parquet('{}/trajectories/00000-517-b88240ee-2707-4e1f-ba89-58fe763ea37c-00001.parquet'.format(save_path_map))
uid_counter = Counter(df_traj['user_id'])
sorted_uids = sorted(uid_counter.keys())
uid_mapping = {uid: i for i, uid in enumerate(sorted_uids)} 
df_traj['user_id'] = df_traj['user_id'].apply(lambda x: uid_mapping[x]) 

In [43]:
a = Counter(df_traj['trip_id'])
#len(a.keys())# 10000
groups = df_traj.groupby(df_traj.trip_id)# len(groups) 10000
traj_dict = defaultdict()
for key in tqdm(a.keys()):
    traj_dict[key] = groups.get_group(key)
    traj_dict[key] = traj_dict[key].sort_values(by=['timestamp'],ascending=[True]).reset_index(drop=True)
    uid_value = traj_dict[key]['user_id'][0]
    traj_dict[key] = traj_dict[key].drop(columns=['user_id'])
    
    traj_dict[key]['timestamp_long'] = traj_dict[key]['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    traj_dict[key]['timestamp_long'] = pd.to_datetime(traj_dict[key]['timestamp_long'])
    time_pre = pd.DataFrame()
    time_pre['timestamp_long'] = pd.date_range(start=traj_dict[key]['timestamp_long'][0], end=traj_dict[key]['timestamp_long'][len(traj_dict[key])-1], freq='1S')
    new_df = pd.merge(time_pre,traj_dict[key], on='timestamp_long',how='outer')#
    new_df  = new_df.groupby(['timestamp_long']).mean().reset_index()
    new_df[["latitude","longitude","speed"]] = new_df[["latitude","longitude","speed"]].interpolate()
    traj_dict[key] = new_df
    traj_dict[key]['timestamp'] = traj_dict[key]['timestamp_long'].apply(lambda x: int(x.timestamp()))
    traj_dict[key]['timestamp'] = traj_dict[key]['timestamp'].astype(int)
    
    uid = {'uid': [uid_value] * len(traj_dict[key])}  
    traj_dict[key]['uid'] = pd.DataFrame(uid)
    traj_dict[key]['speed'] = traj_dict[key]['speed']/1000 # km/s

100%|██████████| 8188/8188 [03:45<00:00, 36.35it/s]


In [44]:
traj_dict = {int(key): value for key, value in traj_dict.items() if len(traj_dict[key]) != 1}

In [45]:
len(traj_dict)

8183

In [46]:
np.save('{}/traj_dict.npy'.format(save_path_map),traj_dict)
print('save well')

save well


In [49]:
len(traj_dict)

8183

## 3.2 decide a area

In [ ]:
min_longitude = float('inf')
max_longitude = float('-inf')
min_latitude = float('inf')
max_latitude = float('-inf')


for df in traj_dict.values():
    min_longitude = min(min_longitude, df['longitude'].min())
    max_longitude = max(max_longitude, df['longitude'].max())
    min_latitude = min(min_latitude, df['latitude'].min())
    max_latitude = max(max_latitude, df['latitude'].max())

## 3.3  generate traj_dict_test

In [50]:
traj_dict= np.load('{}/traj_dict.npy'.format(save_path_map),allow_pickle=True).item()
print(len(traj_dict))
print(list(traj_dict.keys())[:20])

8183
[1501499985030737920, 1504333203681460224, 3951405212996569, 1508971710441996288, 1510528269332660224, 3956605073928317, 1518784835257253888, 1519483109639340032, 3963699746525458, 1531114749079732224, 1542142463291092992, 3964042283025088, 3965059453236096, 3966840040894762, 3944329632330256, 1500743840020230144, 3951917441896828, 3952338276529107, 3953841766114067, 1506916200179974144]


In [51]:
keys_list= list(traj_dict.keys())[:10]
traj_dict_test = {key:traj_dict[key] for key in keys_list}
len(traj_dict_test)
np.save('{}/traj_dict_test.npy'.format(save_path_map),traj_dict_test) # 注意带上后缀名

In [52]:
traj_dict[1501499985030737920]

,timestamp_long,speed,longitude,latitude,timestamp,uid
0,2022-03-09 10:07:46,0.00451,120.141972,33.371805,1646820466,346
1,2022-03-09 10:07:47,0.00463,120.141913,33.371795,1646820467,346
2,2022-03-09 10:07:48,0.00536,120.141864,33.371785,1646820468,346
3,2022-03-09 10:07:49,0.00561,120.141818,33.371763,1646820469,346
4,2022-03-09 10:07:50,0.00555,120.141771,33.371744,1646820470,346
...,...,...,...,...,...,...
1798,2022-03-09 10:37:44,0.00000,120.136199,33.345900,1646822264,346
1799,2022-03-09 10:37:45,0.00000,120.136216,33.345829,1646822265,346
1800,2022-03-09 10:37:46,0.00000,120.136233,33.345759,1646822266,346
1801,2022-03-09 10:37:47,0.00000,120.136250,33.345688,1646822267,346


# 3.4  split data

In [2]:
def split_traj(traj_dict,traj_dict_name):

    total_keys = len(traj_dict)
    sub_dict_size = total_keys // 

    sub_dicts = []
    for i in range(0, total_keys, sub_dict_size):
        sub_dict = {k: traj_dict[k] for k in list(traj_dict.keys())[i:i + sub_dict_size]}
        sub_dicts.append(sub_dict)
    
    for i in tqdm(range(len(sub_dicts))):
        traj_dict_split = sub_dicts[i]
        np.save('{}/{}_{}.npy'.format(save_path_map,traj_dict_name,i),traj_dict_split)

    print('split well')

In [3]:
traj_dict = np.load('{}/traj_dict.npy'.format(save_path_map),allow_pickle=True).item()
split_traj(traj_dict,'traj_dict')

100%|██████████| 41/41 [00:02<00:00, 15.04it/s]

split well
